In [4]:
### Initialize
import XLSX, ExcelFiles, DataTables, DataFrames, CSV, Plots

In [5]:
##Reading in Data
using CSV, DataFrames
Data = CSV.read("Data.csv", DataFrame, normalizenames=true)


,hour,heating_kW_th_h_m2_,PV_electricity_kW,Wind_electricity_kW,Demand_kW
,Float64?,Float64?,Float64?,Float64?,Float64?
1,1.0,0.0,0.0,0.0839,1664.74
2,2.0,0.0001,0.0,0.0839,1551.21
3,3.0,0.0,0.0,0.0839,1429.02
4,4.0,0.0012,0.0,0.0839,1350.95
5,5.0,0.0081,0.0,0.0839,1324.06
6,6.0,0.009,0.0,0.0839,1296.71
7,7.0,0.0139,0.0,0.0839,1597.59
8,8.0,0.0194,0.0,0.1533,1479.02
9,9.0,0.0171,0.0,0.1533,1791.12


In [14]:
ElecDemand = Data[1:8760,5] #kW
PV = Data[1:8760,3] #kW
Cost_Elec = Data[1:8760,6]
ElecDemand = identity.(ElecDemand)
PV = identity.(PV)
Cost_Elec = identity.(Cost_Elec)


8760-element Array{Float64,1}:
 0.0689
 0.065
 0.0603
 0.05
 0.0477
 0.048
 0.0498
 0.0551
 0.0448
 0.0496
 0.0554
 0.0626
 0.0574
 ⋮
 0.0386
 0.0379
 0.038
 0.0396
 0.0423
 0.0461
 0.0477
 0.046
 0.0422
 0.0397
 0.0389
 0.0372

Long Term Variables (Investment Variables) :
E_Rated = Battery Rated Capacity
P_Rated = Battery Rated Power
Binary 1: Invest in Battery
Binary 2: Don't Invest in Battery

Short Term Variables (Operational Variables):
EState_Batt = Battery State of Charge at Time t
PState_Batt = Battery Power at Time t
P_Grid = Power Consumed/ Injected to the grid

Parameters

Cost_BattE = Investment Cost of Battery, energy component

Cost_BattP = Investment cost of battery, power 

Cost_OM= Fixed yearly operation and maintenance costs

DiscountRate = discount rate, werighted average cost of capital

Cost_Electricity = Price of Electricity (optional binary)


In [24]:
BatteryMaxCap = 1.5E3*3600 #kWh
RatedPower = 1.5E3 #kW

1500.0

In [15]:
Cost_Elec

8760-element Array{Float64,1}:
 0.0689
 0.065
 0.0603
 0.05
 0.0477
 0.048
 0.0498
 0.0551
 0.0448
 0.0496
 0.0554
 0.0626
 0.0574
 ⋮
 0.0386
 0.0379
 0.038
 0.0396
 0.0423
 0.0461
 0.0477
 0.046
 0.0422
 0.0397
 0.0389
 0.0372

In [33]:
using JuMP, Gurobi

model = Model(Gurobi.Optimizer)
@variable(model,P_Batt[1:8760])
@variable(model,P_Grid[1:8760])
@variable(model,E_Batt[1:8760])

@objective(model, Min, sum(P_Grid[i]*Cost_Elec[i]) for i in 1:8760)

@constraint(model,[i = 1:8760],ElecDemand[1:8760] == PV[1:8760]+P_Batt[1:8760]+P_Grid[1:8760])
 # + if pulling from supply, ex P_Batt = +5 means taking 5kW from battery -5 would be saving 5kw
@constraint((model),E_Batt[1] == BatteryMaxCap)
@constraint(model, [ i=1:8760], P_Grid[i]*3600 <= E_Batt[i]) ##check indexing
@constraint(model, [ i=1:8760], E_Batt[i+1] == E_Batt[i]+P_Grid[i]*3600 for i in 1:8759)
@constraint(model, [ i=1:8760], P_Batt[i] < RatedPower)

Academic license - for non-commercial use only - expires 2021-08-04


LoadError: [91mThe objective function `Base.Generator{UnitRange{Int64},var"#37#38"}(var"#37#38"(), 1:8760)` is not supported by JuMP.[39m

In [ ]:
Cost_BattE = 300
#300 euro / kWh from Excel Sheet
Cost_BattP = 300
#300 euro/ kW from Excel Sheet
Cost_OM = 20.5 * E_Rated
#20.5 euro / kWh / Yr from Excel Sheet
Discount_Rate = .0025 
## 0.25% ? CHECK THIS
Cost_Elec = Data[1:8760,6]
# from commodity prices on excel -- eur/kWh Elec

In [ ]:

@constraint(P_Batt[1:8760]*3600 < BatteryMaxCap) ##StateOfChargeArray[1:8760] ##check indexing
    




optimize!(model)


PowerDifference = Production - Demand 	kW
Energy Difference = Power Difference * 3600

Option 1: NEG -- Pull Power from the Battery
Option 2: NEG -- Buy Pwoer from the Grid
Option 3: POS -- Save Power to the Battery
Option 4: POS -- Sell Power to the Grid

Objective Function is to minimize electricity costs
1) Battery Investment Costs
(Battery Rated Capacity * Cost of Battery Energy) + (Battery Rated Power * Cost of Battery Power)

2) Present Value of Operation & Maintenance Costs for Battery
SUM for every year -- the investment horizon
    (Battery Rated Capacity * Fixed Yearly O&M Costs)/ (1+discountrate)^Year
    
3) Residual Value of Battery at the end of horizon Y (Depreciation or Energy???)

4) Costs/ Revenue from Buying & Selling to Market
SUM for every year -- 
    Sum for every hour (1:8760)
        (Price of Electricity * Power to/from Grid) / (1+discountrate)^year
Note: This assumes that electricity is bouht and sold at the same price. 
RUN A BINARY SIMLULATION LATER TO DEAL WITH THIS